In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline


/home/linghao/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('/home/linghao/Datasets/Kaggle_SMS/spam.csv',delimiter=',',encoding='latin-1')
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
print(df.head())


     v1                                                 v2
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [4]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [5]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [6]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [7]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 3788 samples, validate on 948 samples
Epoch 1/10


 128/3788 [>.............................] - ETA: 1:56 - loss: 0.6937 - acc: 0.4531

 256/3788 [=>............................] - ETA: 58s - loss: 0.6833 - acc: 0.6562 

 384/3788 [==>...........................] - ETA: 38s - loss: 0.6691 - acc: 0.7292

 512/3788 [===>..........................] - ETA: 29s - loss: 0.6429 - acc: 0.7656

...

3328/3788 [=========================>....] - ETA: 0s - loss: 0.0323 - acc: 0.9916

3456/3788 [==========================>...] - ETA: 0s - loss: 0.0321 - acc: 0.9916

3584/3788 [===========================>..] - ETA: 0s - loss: 0.0316 - acc: 0.9916

3712/3788 [============================>.] - ETA: 0s - loss: 0.0311 - acc: 0.9916

3788/3788 [==============================] - 5s 1ms/step - loss: 0.0307 - acc: 0.9918 - val_loss: 0.0494 - val_acc: 0.9863


In [8]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [9]:
accr = model.evaluate(test_sequences_matrix,Y_test)

 32/836 [>.............................] - ETA: 1s

 96/836 [==>...........................] - ETA: 1s

160/836 [====>.........................] - ETA: 0s

224/836 [=======>......................] - ETA: 0s

288/836 [=========>....................] - ETA: 0s

...

704/836 [========================>.....] - ETA: 0s

768/836 [==========================>...] - ETA: 0s

800/836 [===========================>..] - ETA: 0s

832/836 [============================>.] - ETA: 0s

836/836 [==============================] - 1s 2ms/step


In [10]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.092
  Accuracy: 0.975
